# Dead Tree Dataset Preprocessing

In [26]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("meteahishali/aerial-imagery-for-standing-dead-tree-segmentation")

print("Path to dataset files:", path)

Path to dataset files: /Users/evanchan19/.cache/kagglehub/datasets/meteahishali/aerial-imagery-for-standing-dead-tree-segmentation/versions/1


In [27]:
import os
import shutil
import random
from PIL import Image
import numpy as np
import pandas as pd

# Relative path
dataset_dir = "data/USA_segmentation"
rgb_dir = os.path.join(dataset_dir, "RGB_images")
nrg_dir = os.path.join(dataset_dir, "NRG_images")
mask_dir = os.path.join(dataset_dir, "masks")

# create train/val 
for split in ["train", "val"]:
    for subfolder in ["RGB_images", "NRG_images", "masks"]:
        os.makedirs(os.path.join(dataset_dir, split, subfolder), exist_ok=True)

def build_core_map(directory, prefix):
    return {
        f[len(prefix):]: f
        for f in os.listdir(directory)
        if f.startswith(prefix) and f.lower().endswith((".png", ".jpg", ".jpeg"))
    }

rgb_map = build_core_map(rgb_dir, "RGB_")
nrg_map = build_core_map(nrg_dir, "NRG_")
mask_map = build_core_map(mask_dir, "mask_")


matched_core_names = sorted(list(set(rgb_map) & set(nrg_map) & set(mask_map)))
random.seed(42)
random.shuffle(matched_core_names)

# 80 20 Train Test set 
split_index = int(len(matched_core_names) * 0.8)
train_cores = matched_core_names[:split_index]
val_cores = matched_core_names[split_index:]

# Copy to its direcotry 
def copy_from_map(core_list, split):
    for core in core_list:
        shutil.copy(os.path.join(rgb_dir, rgb_map[core]), os.path.join(dataset_dir, split, "RGB_images", rgb_map[core]))
        shutil.copy(os.path.join(nrg_dir, nrg_map[core]), os.path.join(dataset_dir, split, "NRG_images", nrg_map[core]))
        shutil.copy(os.path.join(mask_dir, mask_map[core]), os.path.join(dataset_dir, split, "masks", mask_map[core]))

copy_from_map(train_cores, "train")
copy_from_map(val_cores, "val")


invalid_masks = []
def validate_masks(mask_folder):
    for fname in os.listdir(mask_folder):
        try:
            img = Image.open(os.path.join(mask_folder, fname)).convert("L")
            arr = np.array(img)
            if arr.ndim != 2 or not np.isin(arr, [0, 255]).all():
                invalid_masks.append(os.path.join(mask_folder, fname))
        except Exception:
            invalid_masks.append(os.path.join(mask_folder, fname))

validate_masks(os.path.join(dataset_dir, "train", "masks"))
validate_masks(os.path.join(dataset_dir, "val", "masks"))


if invalid_masks:
    print("\n❌ Invalid mask files (non-binary or wrong format):")
    for f in invalid_masks:
        print(" -", f)
else:
    print("\n✅ All masks valid: grayscale + binary (0/255)")



✅ All masks valid: grayscale + binary (0/255)


## 2.   Image analysis before training

In [28]:
import os
import cv2
import numpy as np

def analyze_image_sizes(image_dir):
    heights, widths = [], []
    image_files = sorted([
        f for f in os.listdir(image_dir)
        if f.lower().endswith(('.png', '.jpg', '.jpeg'))
    ])

    for fname in image_files:
        path = os.path.join(image_dir, fname)
        img = cv2.imread(path)
        if img is None:
            print(f"⚠️ Failed to load: {fname}")
            continue
        h, w = img.shape[:2]
        heights.append(h)
        widths.append(w)

    heights = np.array(heights)
    widths = np.array(widths)

    print(f"✅ Scanned {len(heights)} images in: {image_dir}")
    print(f"- Average size: {np.mean(widths):.1f} x {np.mean(heights):.1f}")
    print(f"- Min size: {np.min(widths)} x {np.min(heights)}")
    print(f"- Max size: {np.max(widths)} x {np.max(heights)}")


# 替换为你要分析的目录（RGB 或 NRG）
image_dir = "/Users/evanchan19/Desktop/COMP9517/project/data/USA_segmentation/train/RGB_images"
analyze_image_sizes(image_dir)


✅ Scanned 355 images in: /Users/evanchan19/Desktop/COMP9517/project/data/USA_segmentation/train/RGB_images
- Average size: 395.6 x 385.1
- Min size: 317 x 297
- Max size: 630 x 636


In [29]:
import os
import cv2
import numpy as np

def analyze_rgb_nir_pair_sizes(rgb_dir, nrg_dir):
    mismatched = []
    matched = 0
    total = 0
    nir_heights, nir_widths = [], []

    core_names = [
        f.replace("RGB_", "")
        for f in os.listdir(rgb_dir)
        if f.startswith("RGB_") and f.lower().endswith(('.png', '.jpg', '.jpeg'))
    ]

    for core in sorted(core_names):
        rgb_path = os.path.join(rgb_dir, f"RGB_{core}")
        nrg_path = os.path.join(nrg_dir, f"NRG_{core}")

        if not os.path.exists(nrg_path):
            print(f"❌ NRG image missing: {nrg_path}")
            continue

        rgb_img = cv2.imread(rgb_path)
        nrg_img = cv2.imread(nrg_path)

        if rgb_img is None or nrg_img is None:
            print(f"⚠️ Failed to load: {core}")
            continue

        total += 1
        if rgb_img.shape[:2] != nrg_img.shape[:2]:
            mismatched.append(core)
            print(f"❗ Size mismatch in: {core}")
        else:
            matched += 1
            h, w = nrg_img.shape[:2]
            nir_heights.append(h)
            nir_widths.append(w)

    print(f"\n✅ Total checked: {total}")
    print(f"✅ Matched sizes: {matched}")
    if mismatched:
        print(f"❌ Mismatched sizes ({len(mismatched)}):")
        for name in mismatched:
            print(f" - {name}")
    else:
        print("🎉 No size mismatches found between RGB and NRG pairs.")

    # 打印 NIR 图像尺寸统计信息
    if nir_heights and nir_widths:
        nir_heights = np.array(nir_heights)
        nir_widths = np.array(nir_widths)
        print(f"\n📐 NRG image size summary:")
        print(f"- Average size: {np.mean(nir_widths):.1f} x {np.mean(nir_heights):.1f}")
        print(f"- Min size: {np.min(nir_widths)} x {np.min(nir_heights)}")
        print(f"- Max size: {np.max(nir_widths)} x {np.max(nir_heights)}")

# 用你的路径替换以下目录
rgb_dir = "/Users/evanchan19/Desktop/COMP9517/project/data/USA_segmentation/train/RGB_images"
nrg_dir = "/Users/evanchan19/Desktop/COMP9517/project/data/USA_segmentation/train/NRG_images"
analyze_rgb_nir_pair_sizes(rgb_dir, nrg_dir)



✅ Total checked: 355
✅ Matched sizes: 355
🎉 No size mismatches found between RGB and NRG pairs.

📐 NRG image size summary:
- Average size: 395.6 x 385.1
- Min size: 317 x 297
- Max size: 630 x 636


In [4]:
import h5py

def inspect_maskrcnn_weights(h5_path):
    """
    Inspect a Mask R-CNN .h5 weight file to detect:
    - The number of input channels in the first convolutional layer
    - The backbone type: resnet50 or resnet101
    """
    try:
        with h5py.File(h5_path, 'r') as f:
            conv1_found = False
            input_channels = None
            backbone_type = None

            print("🔍 Searching for conv1/kernel:0 and backbone identity...")

            for top_key in f.keys():
                # Optional: print all top keys for debugging
                # print(f"- {top_key}")

                # Check conv1
                if "conv1" in top_key.lower():
                    try:
                        kernel = f[top_key]["conv1"]["kernel:0"]
                        shape = kernel.shape
                        if len(shape) == 4:
                            _, _, in_c, out_c = shape
                            input_channels = in_c
                            conv1_found = True
                            print(f"✅ Found conv1: input channels = {in_c}, output channels = {out_c}")
                    except Exception:
                        continue

                # Check backbone
                if "res5c_branch2a" in top_key or "res4b22_branch2a" in top_key:
                    backbone_type = "resnet101"
                elif "res5a_branch2a" in top_key or "res4f_branch2a" in top_key:
                    backbone_type = "resnet50"

            # Summary
            if not conv1_found:
                print("❌ conv1/kernel:0 not found. Possibly excluded during save or replaced.")
            if backbone_type:
                print(f"✅ Detected backbone: {backbone_type}")
            else:
                print("❓ Could not confidently determine backbone type. Possibly custom or partial save.")

            return input_channels, backbone_type

    except Exception as e:
        print(f"❌ Error reading HDF5 file: {e}")
        return None, None

h5_path = "/Users/evanchan19/Desktop/COMP9517/project/logs/usa_rgb20250730T2338/mask_rcnn_usa_rgb_0011.h5"
inspect_maskrcnn_weights(h5_path)

🔍 Searching for conv1/kernel:0 and backbone identity...
❌ conv1/kernel:0 not found. Possibly excluded during save or replaced.
✅ Detected backbone: resnet101


(None, 'resnet101')

In [6]:
path = "/Users/evanchan19/Desktop/COMP9517/project/model/usa_rgb_nrg_resnet101/mask_rcnn_usa_rgb_0015.h5"
from mrcnn.config import Config
import mrcnn.model as modellib
import numpy as np

class InferenceConfig(Config):
    NAME = "usa_rgb"
    NUM_CLASSES = 1 + 1  # Background + dead tree
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
    DETECTION_MIN_CONFIDENCE = 0.1
    IMAGE_CHANNEL_COUNT = 4
    MEAN_PIXEL = np.array([123.7, 116.8, 103.9, 114.5])
inference_config = InferenceConfig()

# Step 0: 创建 inference model
inference_model = modellib.MaskRCNN(mode="inference", config=inference_config, model_dir="./logs")
inference_model.load_weights(path, by_name=True)

for layer in inference_model.keras_model.layers:
    print(layer.name)

input_image
zero_padding2d_3
conv1
bn_conv1
activation_149
max_pooling2d_3
res2a_branch2a
bn2a_branch2a
activation_150
res2a_branch2b
bn2a_branch2b
activation_151
res2a_branch2c
res2a_branch1
bn2a_branch2c
bn2a_branch1
add_67
res2a_out
res2b_branch2a
bn2b_branch2a
activation_152
res2b_branch2b
bn2b_branch2b
activation_153
res2b_branch2c
bn2b_branch2c
add_68
res2b_out
res2c_branch2a
bn2c_branch2a
activation_154
res2c_branch2b
bn2c_branch2b
activation_155
res2c_branch2c
bn2c_branch2c
add_69
res2c_out
res3a_branch2a
bn3a_branch2a
activation_156
res3a_branch2b
bn3a_branch2b
activation_157
res3a_branch2c
res3a_branch1
bn3a_branch2c
bn3a_branch1
add_70
res3a_out
res3b_branch2a
bn3b_branch2a
activation_158
res3b_branch2b
bn3b_branch2b
activation_159
res3b_branch2c
bn3b_branch2c
add_71
res3b_out
res3c_branch2a
bn3c_branch2a
activation_160
res3c_branch2b
bn3c_branch2b
activation_161
res3c_branch2c
bn3c_branch2c
add_72
res3c_out
res3d_branch2a
bn3d_branch2a
activation_162
res3d_branch2b
bn3d_bra